# Running simulations

The basic functionality for performing the AWS simulations is implemented by the `aws` package. With this in place, running simulations is a matter of setting up the simulations with the desired parameters and then executing them via IPyParallel.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import ipyparallel as ipp
import os
import glob
from aws.retrieval import Simulation, Retrieval
from aws.data import Profiles
from aws.sensor import ICI
from aws import aws_path
import random

## Connecting to IPyParallel

Assuming that the IPyParallel controller, hub and engines are running we create a client, which holds the connection to the controller.

In [2]:
from aws.retrieval import Retrieval
client = ipp.Client(profile="gold")
view = client.load_balanced_view()

Since we are already running 4 engines on each of the gold machines, which only have 4 cores, we set the `OMP_NUM_THREADS` environment variable to 1 on all engines to avoid ARTS from launching to many threads.

Note that this notebook is running on your local machine, not on the gold cluster. To execute a cell on the engines we need to use
the  `%%px` cell magic, which executes the cell on the remote engines instead on the local machine.

In [3]:
%%px
%env OMP_NUM_THREADS=1

[stdout:0] env: OMP_NUM_THREADS=1
[stdout:1] env: OMP_NUM_THREADS=1
[stdout:2] env: OMP_NUM_THREADS=1
[stdout:3] env: OMP_NUM_THREADS=1
[stdout:4] env: OMP_NUM_THREADS=1
[stdout:5] env: OMP_NUM_THREADS=1
[stdout:6] env: OMP_NUM_THREADS=1
[stdout:7] env: OMP_NUM_THREADS=1
[stdout:8] env: OMP_NUM_THREADS=1
[stdout:9] env: OMP_NUM_THREADS=1
[stdout:10] env: OMP_NUM_THREADS=1
[stdout:11] env: OMP_NUM_THREADS=1
[stdout:12] env: OMP_NUM_THREADS=1
[stdout:13] env: OMP_NUM_THREADS=1
[stdout:14] env: OMP_NUM_THREADS=1
[stdout:15] env: OMP_NUM_THREADS=1
[stdout:16] env: OMP_NUM_THREADS=1
[stdout:17] env: OMP_NUM_THREADS=1
[stdout:18] env: OMP_NUM_THREADS=1
[stdout:19] env: OMP_NUM_THREADS=1
[stdout:20] env: OMP_NUM_THREADS=1
[stdout:21] env: OMP_NUM_THREADS=1
[stdout:22] env: OMP_NUM_THREADS=1
[stdout:23] env: OMP_NUM_THREADS=1


## Setting up the simulation

The next step is to setup the simulation with the desired configuration.

## Choosing the sensor

Currently two sensor configurations are available:

  * `ATMS`: Simplified version of the upper channels of the
     ATMS sensor.
  * `AWS`: The channel setup to use for the full simulations.

In [5]:
def simulate_orbit(filename):
    import os
    import numpy as np
    import os
    from aws.retrieval import Simulation, Retrieval
    from aws.data import Profiles
    from aws.sensor import ICI
    from aws import aws_path
    
    
    # Simulation settings
    
#    sensor = ATMS()
#    sensor = AWS()
    sensor = ICI()
    ice_shape = np.random.choice(["Perpendicular3BulletRosette",
                                  "LargeColumnAggregate",
                                  "LargePlateAggregate"])
    ice_shape_name = ice_shape.rjust(30)
    
    # Input data
    
    data_provider = Profiles(filename)
    retrieval = Retrieval(data_provider, ice_shape)
    data_provider.ice_shape = ice_shape_name
    
    # Setup the simulation
    
    simulation = Simulation(sensor, retrieval, ice_shape)
    
    # Output data
    
    inputs = [("ice_shape", ("name_length",)),] # List of inputs to store in results
    name, _ = os.path.splitext(os.path.basename(filename))
    output_file = os.path.join(os.path.dirname(filename), "..", "ICI_m60_p60", name + ".nc")
#    output_file = os.path.join(os.path.dirname(filename), "..", "ATMS", name + ".nc")
#    path = os.path.expanduser("~/aws/data")
#    output_file = os.path.join(path, name + ".nc")
    simulation.initialize_output_file(output_file,
                                     [("cases", -1, 0)], # Name of dimension for different simulations
                                     inputs=inputs)
    # Loop over profiles in file
    simulation.setup()
    
    append = True
    if append and "cases" in simulation.output_file.dimensions:
        n = simulation.output_file.dimensions["cases"]
        simulation.run_ranges(range(n, data_provider.n_profiles))
    else:
        simulation.run_ranges(range(data_provider.n_profiles))

## Execute on view

In [6]:
files = glob.glob(os.path.expanduser("~/Dendrite/Projects/AWS-325GHz/Cases_m60_p60/c_of_2015_*.mat"))
#random.shuffle(files)
# Try for 24 files for now
#files = glob.glob(os.path.expanduser("~/Dendrite/Projects/AWS-325GHz/AWS/*clearsky.nc"))
#files = [files[i].replace('AWS', 'CasesV1') for i in range(len(files))]
#files = [files[i].replace('_clearsky.nc', '.mat') for i in range(len(files))]
#print(files)
with open('shuffled_files.txt', 'w') as f:
        for file in files:
            f.write("%s\n" % file)
#with open('shuffled_files.txt', 'r') as f:
#    files = f.readlines()     
#files    

In [7]:
results = view.map(simulate_orbit, files[:48], block=False)

In [11]:
results.status


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'error',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'error',
 'error',
 'error',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Run clearsky simulations

In [ ]:
def simulate_orbit_clearsky(filename):
    import os
    import numpy as np
    import os
    from aws.retrieval import Simulation, Retrieval
    from aws.data import Profiles
    from aws.sensor import ATMS, AWS
    from aws import aws_path
    
    
    # Simulation settings
    
#    sensor = ATMS()
    sensor = AWS()
    ice_shape = np.random.choice(["Perpendicular3BulletRosette",
                                  "LargeColumnAggregate",
                                  "LargePlateAggregate"])
    ice_shape_name = ice_shape.rjust(30)
    
    # Input data
    
    data_provider = Profiles(filename)
    retrieval = Retrieval(data_provider, ice_shape)
    data_provider.ice_shape = ice_shape_name
    
    # Setup the simulation
    
    simulation = Simulation(sensor, retrieval, ice_shape)
    
    # Output data
    
    inputs = [("ice_shape", ("name_length",)),] # List of inputs to store in results
    name, _ = os.path.splitext(os.path.basename(filename)) 
#    output_file = os.path.join(os.path.dirname(filename), "..", "ATMS", name + "_clearsky.nc")
    output_file = os.path.join(os.path.dirname(filename), "..", "AWS_m60_p60", name + "_clearsky.nc")
    simulation.initialize_output_file(output_file,
                                     [("cases", -1, 0)], # Name of dimension for different simulations
                                     inputs=inputs)
    # Loop over profiles in file
    simulation.setup()

    append = True
    if append and "cases" in simulation.output_file.dimensions:
        n = simulation.output_file.dimensions["cases"]
        simulation.run_ranges(range(n, data_provider.n_profiles), clearsky = True)
    else:
        simulation.run_ranges(range(data_provider.n_profiles), clearsky = True)
        
#    append = True
#    if append:
#        n = simulation.output_file.dimensions["cases"]
#        simulation.run_ranges(range(n, data_provider.n_profiles),  clearsky=True)
#    else:
#        simulation.run_ranges(range(data_provider.n_profiles), "wb",  clearsky=True)
    

Using same files as for allsky case

In [ ]:
#files = glob.glob(os.path.expanduser("~/Dendrite/Projects/AWS-325GHz/Cases_m60_p60/c_of_2015_204_1?.mat"))
#random.shuffle(files)
results_cs = view.map(simulate_orbit_clearsky, files[:24], block=False)

In [ ]:
results_cs.status